# 3. Implementing Attention Mechanism

We would cover the following:
* Why do we need attention mechanisms in neural networK
* Implemetin a basic self-attention framework, progressing to an enhanced self-attention mechanism
* A causal attention module that allows LLMs to generate on token at a time
* Masking randomly selected attention weights with dropouts to reduce overfitting
* Stacking multiple causal attention modules into a multi-head attention module

### Type oF Attention mechanis
* Simplifed Self-attention
* Self-attention
* Causal attention
* Multi-head attention

## 3.2 Capuring Data Dependencies wity Attention Mechanism
Initialy the Bahdanau mechanism was used with RNNs to give the current layer access to previous layers

**Self-attention** - This mechanism allows each position in the input sequence to consider the relevancy of (or attend
to) all other positions in the same sequence when computing the representation of the sequence

## 3.3 Attending to Various Parts of the Input
The self refers to the ability of the llm refer to weights of other tokens in the same sequence.
The goal of self-attention is to compute the **context vector** for each each input element that combines information
from all other inputs \
Take note of the following terms:
* input elements x
* attention scores - this is vector of how the element of each input attents to all other elements in the input sequence
* attention weights - this is normalized attention weights
* context vector - this a vector derive by combining each input vector with it's attention weights

### How to calculate attention score 
We will calculate attention score by of the second element with reference to every other elements and this is calculated
taking the dot product of the current element (query token) and the every other element.
Then we would have a set of scores that would make up the context vector

# Calculating the attention scores
attention_scores_2 = torch.empty(size=(inputs.shape[0],)) # We create an empty tensor with dimention same as that of the inputs
for index, query in enumerate(inputs): # We iterate throught he inputs (each embedding vector in the input)
    attention_scores_2[index] = torch.dot(query, inputs[index]) # for each input, we find the dot product of the query and the current input and store it at the index position of the attention_scores
print(attention_scores_2) # print the attention scores

**Dot Product** \
A dot product is anothe way to multiply two vectors element-wise and then summing the results

In [ ]:
attention_scores2 = torch.empty(inputs.shape[0])
query = inputs[0]
result = 0
for index, item in enumerate(query):
    result = result + query[index] * item
print(result)
print(torch.dot(inputs[0], query))

In [ ]:
# Calculating the attention matrix
# This is a matrix of the attention scores of all the inputs
attention_scores_matrix = torch.empty(inputs.shape[0], inputs.shape[0])
for index, query in enumerate(inputs):
    attention_scores_matrix[index] = torch.matmul(inputs, query)
print(attention_scores_matrix)

In [ ]:
# We can also obtain the attention_scores_matrix by mulitplying the 
print(torch.matmul(inputs, inputs.T))
print(inputs @ inputs.T)

In [ ]:
# Context vector calculation for the input "Your journey starts with one step"
import torch
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your    (x^1)
        [0.55, 0.87, 0.66], # journey (x^2)
        [0.57, 0.85, 0.64], # starts  
        [0.22, 0.58, 0.33], # with
        [0.77, 0.25, 0.01], # one
        [0.05, 0.80, 0.55]  #step
    ]
)

**Dot Product and Similarity** \
The dot product of of two vectors is a measure of similrity because it measures how closely two vectors are aligned.
When the dot product is high, it shows taht theres is a greate degree of similarity between the vectors.
Therefore, in the context of self-attention, it determines the extent to which each element attends to every other element
in the sequence

**Normalization** \
We would have to normilize each of the attention scores we obtained. \
The goal of normalization is to obtain attention weights that sum up to 1
\
**Attention Weights** \
We obtain the attention weigths by normalizing the attention scores. This is achieved by dividing each attention score by the
sum of all the attention scores 

**The Softmax Function** \
Using the softmax function for normalization. This function is more efficient at handling extreme values and is more
gives more favorable gradient properties during training.
It also ensures that teh attention weights are always positive making the output interpretable as probabilities or relative
importance where higher weights indicates greater importance.
Pytorch has an implementation of the softmax() fuction which is much better

In [ ]:
## Softmax implementation
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attention_weights = softmax_naive(attention_scores)
print("Attention weights: ", attention_weights)
print("Attention weights sum: ", attention_weights.sum())

In [ ]:
# Peform normilization
attention_weight_2 = attention_scores_2 / sum(attention_scores_2)
print(attention_weight_2)

## The Context Vector
This is like the enriched embedding vectors.
It incorporates each input vector with the attention weights for that vector.
To obtain this, we must multiply each input vector with the correponding attention weight

In [ ]:
# Computing the context vector
query = inputs[1]
context_vector_2 = torch.zeros(query.shape)
for index, query in enumerate(inputs):
    context_vector_2 = attention_weight_2[index] * query
print(context_vector_2)

In [ ]:
# Normalize the attention scores matrix
attention_weights_matrix = torch.softmax(attention_scores_matrix, dim=1)
print(attention_weights_matrix)

In [ ]:
# check that the rows sum up to 1
for item in attention_weights_matrix:
    print(item.sum())

In [ ]:
# Compute the context vector matrix
context_vector_matrix = torch.matmul(attention_weights_matrix, inputs) # same as attention_weights_matrix @ inputs
print(context_vector_matrix)

## 3.4 Self-Attention With Trainable Weights

To implement the self-attention mechanisme, we would need to introduce the trainable weight matrices
Wq, Wk and Wv
These three matrices are used to project the embedded input tokens into query, key and value vectors respectively /
In the first step of the self-attention mechanism with traninable weights, we compute:
* query (q)
* key (k)
* value (v) \
for the input elements x \

We designate the second input x2 as the query input
* the query vector is obtained by matrix multiplication between the input and the weight matrix Wq
* the key vector is obtained by matrix multiplication between the input and the weight matrix Wk
* the value vector is obtained by a matriv multiplication between the input and weight matrix Wv

In [2]:
import torch

#initialize the inputs
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your    (x^1)
        [0.55, 0.87, 0.66], # journey (x^2)
        [0.57, 0.85, 0.64], # starts  
        [0.22, 0.58, 0.33], # with
        [0.77, 0.25, 0.01], # one
        [0.05, 0.80, 0.55]  #step
    ]
)

x_2 = inputs[1] #the second element of the inputs
d_in = inputs.shape[1] # the size of the input embedding, d=3
d_out = 2 # the output embedding size, d_out=2

In [3]:
print(x_2)

tensor([0.5500, 0.8700, 0.6600])


In [4]:
# Initialize the weight matrices
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
# Note that we set the requires_grad parameter to False, but if we would use the weight matrices for training, we set to True

In [5]:
W_key

Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]])

In [6]:
# We now comput the query, key and value vectors
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

In [7]:
print(query_2)
print(key_2)
score_2 = query_2.dot(key_2)
print(score_2)

tensor([0.4306, 1.4551])
tensor([0.4433, 1.1419])
tensor(1.8524)


## Weight Parameters vs Attention Weights

The weight matrices here is not the same as the attention weights.
The weight parameters of a network are the values that are optimized during the training phase
Attention weights are the values that determine the extent to which the context vector depends on other parts of the input \
Weight parameters are teh fundamental learnt coefficients that define the network structur while attention weights are
dynamic context-specific values

In [8]:
# Obtain all keys and values
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [9]:
keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1543, 0.2674],
        [0.3275, 0.9642]])

In [10]:
keys_2 = keys[1]
keys_2

tensor([0.4433, 1.1419])

In [11]:
# Compute the attention scores for input 2 against keys_2
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [12]:
# Compute the attention scores for the given query
attention_scores_2 = query_2 @ keys.T
print(attention_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.4555, 1.5440])


In [13]:
# We now need to calculate the attention weights
# this is done by scaling the attention scores by dividing them by the square root of the embedding dimention of the keys

d_k = keys.shape[-1] # calculate the size of the last dimension of the keys tensor
attn_weight_2 = torch.softmax(attention_scores_2/ d_k**0.5, dim=-1) # dk**0.5 is the scaling factor and used to stabilize the softmax computation
print(attn_weight_2)

tensor([0.1510, 0.2278, 0.2213, 0.1319, 0.0848, 0.1832])


In [14]:
# Compute the context vector by combining all value vectors via the attention weights
context_vec_2 = attn_weight_2 @ values
print(context_vec_2)

tensor([0.3062, 0.8178])


## Assignment 1: Compute all context vectors

In [15]:
# Solution to Assignment 1
context_vectors = torch.empty(inputs.size(0), W_value.size(1))
for index, item in enumerate(inputs):
    # compute the query
    query = item @ W_query
    key = inputs @ W_key
    value = inputs @ W_value

    # compute the attention scores across the keys
    attention_scores = query @ key.T

    # calculate the attention weight
    key_dim = key.shape[-1]
    attention_weights = torch.softmax(attention_scores/key_dim**0.5, dim=-1)

    # compute the context vector
    context_vector = attention_weights @ value

    # add the computed context vector to postion index of the context_vectors
    context_vectors[index] = context_vector

context_vectors

tensor([[0.2993, 0.8003],
        [0.3062, 0.8178],
        [0.3059, 0.8170],
        [0.2942, 0.7874],
        [0.2906, 0.7782],
        [0.2987, 0.7989]])

In [16]:
# Using our concise self-attention class v1
import torch
from utilities.SelfAttention import SelfAttention_v1
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2993, 0.8003],
        [0.3062, 0.8178],
        [0.3059, 0.8170],
        [0.2942, 0.7874],
        [0.2906, 0.7782],
        [0.2987, 0.7989]], grad_fn=<MmBackward0>)


In [17]:
# Using the improved self-attention class v2
from utilities.SelfAttention import SelfAttention_v2
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0715,  0.0736],
        [-0.0729,  0.0720],
        [-0.0730,  0.0719],
        [-0.0737,  0.0707],
        [-0.0742,  0.0699],
        [-0.0733,  0.0713]], grad_fn=<MmBackward0>)


## Assignment 2
Transfer the weight matrices from an instance of SelfAttention_v1 to and instance of SelfAttention_v2

### Solution to Assignment 2
To transfer the weight matrices from an instance of SelfAttention_v2 to an instance of SelfAttention_v1, you will have the map the weights of the nn.Linear layers in SelfAttention_v2
to the nn.Parameter tensors in SelfAttention_v1

In [18]:
with torch.no_grad():
    sa_v1.W_query.data = sa_v2.W_query.weight.clone().T
    sa_v1.W_key.data = sa_v2.W_keys.weight.clone().T
    sa_v1.W_value.data = sa_v2.W_values.weight.clone().T

In [19]:
output_v1 = sa_v1(inputs)
output_v2 = sa_v2(inputs)
print(output_v1)
print(output_v2)

tensor([[-0.0715,  0.0736],
        [-0.0729,  0.0720],
        [-0.0730,  0.0719],
        [-0.0737,  0.0707],
        [-0.0742,  0.0699],
        [-0.0733,  0.0713]], grad_fn=<MmBackward0>)
tensor([[-0.0715,  0.0736],
        [-0.0729,  0.0720],
        [-0.0730,  0.0719],
        [-0.0737,  0.0707],
        [-0.0742,  0.0699],
        [-0.0733,  0.0713]], grad_fn=<MmBackward0>)


## 3.5 Using Causal Attention (Masked Attention)
This used to ensure that only the tokens that appear prior to the current position are considered
when predicting the next token in the sequence.
This is called causal attention and is a specialized from of self-attention

For each token in the input, we mask out future tokens and restricts access to only the previous and 
current inputs in the sequence. In contrast, the standard self-attention mechanism allows access to the entire
input sequence at once.

* Step 1 - we mask out the attention weights above the diagonal
* Step 2 - we normalize existing attention weights

In [20]:
print("Query from v1")
print(inputs @ sa_v1.W_query)

print("Query from v2") 
print(sa_v2.W_query(inputs))

Query from v1
tensor([[ 0.6600, -0.2047],
        [ 0.9091, -0.4471],
        [ 0.8960, -0.4419],
        [ 0.5034, -0.2633],
        [ 0.3627, -0.2150],
        [ 0.6628, -0.3292]], grad_fn=<MmBackward0>)
Query from v2
tensor([[ 0.6600, -0.2047],
        [ 0.9091, -0.4471],
        [ 0.8960, -0.4419],
        [ 0.5034, -0.2633],
        [ 0.3627, -0.2150],
        [ 0.6628, -0.3292]], grad_fn=<MmBackward0>)


In [49]:
print("Keys from v1")
print(inputs @ sa_v1.W_query)

print("Keys from v2")
print(sa_v2.W_keys(inputs))

Keys from v1
tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2917, 0.5786],
        [0.2568, 1.0533]], grad_fn=<MmBackward0>)
Keys from v2
tensor([[ 0.3147, -0.4016],
        [-0.0298, -0.4459],
        [-0.0170, -0.4262],
        [-0.1054, -0.2724],
        [ 0.1972,  0.0942],
        [-0.2258, -0.4782]], grad_fn=<MmBackward0>)


In [51]:
print("Values from v1")
print(inputs @ sa_v1.W_key)

print("Values from v2")
print(sa_v2.W_values(inputs))

Values from v1
tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1543, 0.2674],
        [0.3275, 0.9642]], grad_fn=<MmBackward0>)
Values from v2
tensor([[-0.0872,  0.0286],
        [-0.1137,  0.0766],
        [-0.1018,  0.0927],
        [-0.0912, -0.0026],
        [ 0.1571,  0.3764],
        [-0.2085, -0.1546]], grad_fn=<MmBackward0>)


In [21]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_keys(inputs)
attention_scores = queries @ keys.T
attention_weights = torch.softmax(attention_scores / keys.shape[-1]**0.5, dim=-1)
print(attention_weights)

tensor([[0.1927, 0.1651, 0.1656, 0.1554, 0.1698, 0.1514],
        [0.2051, 0.1667, 0.1670, 0.1503, 0.1626, 0.1484],
        [0.2045, 0.1667, 0.1670, 0.1505, 0.1626, 0.1487],
        [0.1874, 0.1671, 0.1673, 0.1575, 0.1639, 0.1568],
        [0.1817, 0.1674, 0.1675, 0.1599, 0.1635, 0.1600],
        [0.1941, 0.1669, 0.1671, 0.1547, 0.1637, 0.1534]],
       grad_fn=<SoftmaxBackward0>)


## Obtain Masked Attention Weights
One way to obtain the masked attention weight matrix in a causal attention is to apply the softmax function to the attention scores,
zeroing out the elements above the diagonal and normalizing the resulting matrix.

In [22]:
#Step 1 - We first obtain the weights
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_keys(inputs)
values = sa_v2.W_values(inputs)

attention_scores = queries @ keys.T
attention_weights = torch.softmax(attention_scores / keys.shape[-1]**0.5, dim=-1)
print(attention_weights)

tensor([[0.1927, 0.1651, 0.1656, 0.1554, 0.1698, 0.1514],
        [0.2051, 0.1667, 0.1670, 0.1503, 0.1626, 0.1484],
        [0.2045, 0.1667, 0.1670, 0.1505, 0.1626, 0.1487],
        [0.1874, 0.1671, 0.1673, 0.1575, 0.1639, 0.1568],
        [0.1817, 0.1674, 0.1675, 0.1599, 0.1635, 0.1600],
        [0.1941, 0.1669, 0.1671, 0.1547, 0.1637, 0.1534]],
       grad_fn=<SoftmaxBackward0>)


In [23]:
#Step 2 - Create a mask using the tril function such that the values above the diagonal are zero
context_length = attention_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [24]:
#Step 3 - Multiply the attention weights with this mask to zero out the diagonals
masked_attention_weights = attention_weights * mask_simple
print(masked_attention_weights)

tensor([[0.1927, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2051, 0.1667, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2045, 0.1667, 0.1670, 0.0000, 0.0000, 0.0000],
        [0.1874, 0.1671, 0.1673, 0.1575, 0.0000, 0.0000],
        [0.1817, 0.1674, 0.1675, 0.1599, 0.1635, 0.0000],
        [0.1941, 0.1669, 0.1671, 0.1547, 0.1637, 0.1534]],
       grad_fn=<MulBackward0>)


In [25]:
#Step 4 - Renormalize the masked attention weights
row_sums = masked_attention_weights.sum(dim=-1, keepdim=True)
masked_attention_weights_normalized = masked_attention_weights / row_sums
print(masked_attention_weights_normalized)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2163, 0.1993, 0.1994, 0.1904, 0.1946, 0.0000],
        [0.1941, 0.1669, 0.1671, 0.1547, 0.1637, 0.1534]],
       grad_fn=<DivBackward0>)


### More Efficient Masked Attention Weights
A more efficient way to obtain masked attention weight matrix in causal attention is to mask the attention scores with negative values before applying the softmax function

In [26]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attention_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2005, 0.0850, 0.0855, 0.0203, 0.0513,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.0997, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [38]:
attention_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attention_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2163, 0.1993, 0.1994, 0.1904, 0.1946, 0.0000],
        [0.1941, 0.1669, 0.1671, 0.1547, 0.1637, 0.1534]],
       grad_fn=<SoftmaxBackward0>)


## Masking Additional Weights With Dropout
Could be applied after computing the attention weights
This is done to reduce overfitting

In [40]:
# Example of dropout
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # we choose a dropout rate of 50%
example = torch.ones(6,6)
print(dropout(example))

# Note that in the output, the tensor contains twos and this is because then non-dropped-out elements are scaled by a factor of 1/(1-p) where p is the scaling factor
# In this case the scaling factor is 0.5. So after dropout, the remaining ones are scaled by 1/(1-0.5) = 2
# Therefore, we have twos instead of ones

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [29]:
# Let's now apply dropout to the attention weights matrix
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
attention_weights_do = dropout(attention_weights)
print(attention_weights_do)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4325, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3338, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


In [51]:
# Duplicating the example to create 2 batches
# There would be 2 inputs with 6 tokens each where each token has an embedding dimension of 3
batch = torch.stack((inputs, inputs), dim=0)


torch.Size([2, 6, 3])
3


In [54]:
import torch

#initialize the inputs
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your    (x^1)
        [0.55, 0.87, 0.66], # journey (x^2)
        [0.57, 0.85, 0.64], # starts  
        [0.22, 0.58, 0.33], # with
        [0.77, 0.25, 0.01], # one
        [0.05, 0.80, 0.55]  #step
    ]
)

x_2 = inputs[1] #the second element of the inputs
d_in = inputs.shape[1] # the size of the input embedding, d=3
d_out = 2 # the output embedding size, d_out=2
print(inputs.shape)

torch.Size([6, 3])


In [49]:
# Utilizing the CausalAttention class

from utilities.SelfAttention import CausalAttention
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vectors = ca(batch)
print(context_vector.shape)


torch.Size([2])


## 3.6 Muli-head Attention
This maeas that we divide the attention mechanism into multipoe heads each operating independently.
Multi-head attention could consist of several single-head attention modules stacked on top of each other.
For example, let's say we have two single head attention module. In this case, instead of using a singel matrix Wv for computing the value matrices in a multi-head attention module with two heads, we now have two value weight matrices Wv1 and Wv2. Same applies to Wq and Wk.
This means that we also obtain two sets of vectors Z1 and Z2 that we can combine into a single vector matrix Z.